In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mybiotools as mbt
import pr_peaks as pp
import os, sys

# 2018-07-23 Refreshing my mind
Waiting for the new samples to be processed, I'll look at the code I wrot so far and try to re-obtain the results that I left.

## Simulation results

In [ ]:
n = 100                                         # number of sites in the system
mu = 50                                         # number of searchers in the system
omega_t_initial = pp.init_omega_t(n, mu)        # initial occupancy vector
site_taus = 2.0 * np.ones(n)                    # time spent on each site
Hsites = [10, 20, 30, 40, 50, 60, 70, 80, 90]
site_taus[Hsites] = 20.0                        # define high-affinity sites
H = [[] for i in xrange(n)]                     # contacts between sites
boost = 1.0                                     # boost received for tetramerization

In [ ]:
# define the JumpingModel
uniform = pp.JumpingModel(H, site_taus, boost)

# set the run parameters and run
nsteps = 100000
uniform.run(nsteps, omega_t_initial)

In [ ]:
plt.plot(uniform.theta[mu])

Bug spotted! This means that before refreshing my mind on how the analysis of the experimental data works, I need to find and fix this bug.

I suspect that the origin of this bug is the fact that there is a sort of "race condition". In this variant of the algorithm, there is a deterministic amount of time that each of the searchers spend on each site. Since all the sites have exactly the same amount of time spent, then there are time steps in which many of the searchers need to find another site. The algorithm deals with every site in the system in a progressive way, so the result is that with very high probability the first sites will be occupied: in fact, they are the first ones being freed, so the last searchers always have those sites available.

I'll see if this bug is fixed by a random order of picking the searchers.

In [ ]:
a = np.arange(n)
print np.random.choice(a, size=n, replace=False)

In [ ]:
# define the JumpingModel
uniform = pp.JumpingModel(H, site_taus, boost)

# set the run parameters and run
nsteps = 100000
uniform.run(nsteps, omega_t_initial)

In [ ]:
plt.plot(uniform.theta[mu])

Okay, bug fixed! The simulation time gets considerably slower, but this for the moment is not a big problem.

Now let's perform the simulation for varying number of searchers.

In [ ]:
uniform = pp.JumpingModel(H, site_taus, boost)
mus = np.arange(1, 20, 2)
for mu in mus :
    mbt.log_message('uniform', 'mu = %2d'%(mu))
    omega_t_initial = pp.init_omega_t(n, mu)
    uniform.run(nsteps, omega_t_initial)

In [ ]:
Lsites = [i for i in xrange(n) if i not in Hsites]
pp.H_to_L(uniform, Hsites, Lsites)
plt.plot(mus, uniform.H_to_L)

Now let's put back in the tetramerization boost.

In [ ]:
Hboost = [[] for i in xrange(n)]
Hboost[Hsites[0]] = [Hsites[1]]
Hboost[Hsites[1]] = [Hsites[0]]
boost = 10.0
HH = pp.JumpingModel(Hboost, site_taus, boost)
for mu in mus :
    mbt.log_message('HH', 'mu = %2d'%(mu))
    omega_t_initial = pp.init_omega_t(n, mu)
    HH.run(nsteps, omega_t_initial)

In [ ]:
plt.plot(mus, HH.H_to_L, 'o--', color='b', label='HH model')
plt.plot(mus, uniform.H_to_L, 'o--', color='r', label='Uniform model')
plt.xlabel(r'$\mu$')
plt.ylabel('H to L ratio')
plt.legend(loc='upper right')
plt.show()